<a href="https://colab.research.google.com/github/amf60/Vac_Covid19_BR_SC_Combinations_of_Concern_5/blob/main/Combinations_of_Concern_5_sc02092021_07am.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<Font color = 'brown'>**Covid-19 vaccination records in the Brazilian state of Santa Catarina**

<Font color = 'brown'>**Vac_Covid19_BR_SC_Combinations_of_Concern_5**


*   Clinical Data Scientist: Aldir MEDEIROS FILHO
*   FLORIANOPOLIS, [Santa Catarina(SC)](https://en.wikipedia.org/wiki/Santa_Catarina_(state), Brasil
*   02 Sep 2021


**Introduction**:

This is the 5th monthly edition of my personal project named 'Combinations_of_Concern - CoC'. The first 4 were initiated and developped during my attendance to [Alura Bootcamp Applied Data Science II (Brazilian Winter 2021)] (https://www.alura.com.br/bootcamp/data-science-aplicada/matriculas-abertas)that was done in parallel to a sequence of 32 Alura courses focused exclusively on the Python language between 01 May and 30 Aug 2021.


This 5th edition is a continuation and extension of the exploratory work I already done during the first four editions, when I reviewed the OpenDataSus registers of covid19 vaccination in the Brazilian State of Santa Catarina(SC).

For the first two editions, due to my limited knowledge of Python I used a mix of terminal command line, Python&Colab and Excel, one for each of the 3 phases of the project. From the 3rd edition the learnings from the Alura courses helped to deliver all the 3 phases in one Colab notebook.

The main findings until here were: 

* From February to end June 2021 a constant trend of ~ 12% of registers for which we can not confirm which vaccine an individual received when I cross-checked 3 variables from the master dataset. I named this troublesome combinations as <Font color = 'brow'>'Combinations of Concern' - CoC </Font> (step 5 here after for definition).

* At the end of July we noticed a reduction to ~ 10%  of total of registrations responding to my definition of 'Combination of Concern'.

<font color='blue'>My objective for this 5th edition, is to check to which direction, up or down, the trend of hte proportion of 'Combinations_of_Concern' is heading on.

Before we move ahead, let's just a quick remind how the masters datasets evolved so far plus this fifth edition:

| Edition | 'Combinations_of_Concern" github                                                                                                                   | Published    | raw file        | Raw size | columns | rows (registrations) | new registrations |
|---------|----------------------------------------------------------------------------------------------------------------------------------------------------|--------------|-----------------|----------|---------|----------------------|-------------------|
| 1+2     | https://github.com/amf60/Vac_Covid19_BR_SC_Combinations_of_Concern_1 <br> <br>https://github.com/amf60/Vac_Covid19_BR_SC_Combinations_of_Concern_2 | early Jun 21 | sc3005_05am.csv |  1.11 GB |    34   |            2,017,225 |                   |
| 3       | https://github.com/amf60/Vac_Covid19_BR_SC_Combinations_of_Concern_3                                                                               | early Jul 21 | sc0207_07am.csv |  1.71 GB |    34   |            3.133.344 |         1.116.119 |
| 4       | https://github.com/amf60/Vac_Covid19_BR_SC_Combinations_of_Concern_4                                                                               | early Aug 21 | sc0408_10am.csv |  2.6 GB  |    34   |            4.854.329 |         1.720.985 |
| 5       |                                                                                                                                                    | early Sep 21 | sc0209_07am.csv |  3.66 GB |    34   |            6.822.321 |         1.967.992 |

amf: the table here after was done with the help from this [source](https://www.tablesgenerator.com/markdown_tables)

<font color ='orange'>AMF: Technical Warning: At the end of Augst 2021, due to the significant increase on the rows in the raw dataset(sc0209_07am), I was obliged to subscribe to Google Colab Pro in order to have a connection to GPU and high-RAM runtime available. The two next blocs of code are from Google Colab's instruction on to have access to GPU and high_RAM runtime. (https://colab.research.google.com/notebooks/pro.ipynb#scrollTo=V1G82GuO-tez)

In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# <font color = 'blue'>**Phase 1 - importing master dataset and some data wrangling**

In [5]:
import pandas as pd
import numpy as np
import matplotlib as mpl
#import seaborn as sns

print('Versão do pandas -> %s' % pd.__version__)
print('Versão do numpy -> %s' % np.__version__)
print('Versão do matplotlib -> %s' % mpl.__version__)
#print('Versão do seaborn -> %s' % sns.__version__)
#print('Versão do scipy -> %s' % scipy.__version__)

Versão do pandas -> 1.1.5
Versão do numpy -> 1.19.5
Versão do matplotlib -> 3.2.2


# <Font color = 'green'>**Step 1** - imported the raw master csv dataset into the Google Drive folder created for this project.

This is the fastest process I found to upload file into Colab (less than 4 minutes for a 3.6 GB master file).

However when the file was inside the drive folder, when right clicked on it, I could not find the path to this file using pd.read_csv(''). 

To resolve it I used tips from this [source](https://neptune.ai/blog/google-colab-dealing-with-files). That helped me transfer the file from the drive folder to Colab Files, as shown in the Alura teachings, and then to copy path to add to pd.read_csv.

# <Font Color = 'green'>**Step 2** - Imported the raw master file into Google Colab, renamed the headers and some data cleaning

Renamed all 34 original colum's head & variable names (shorter and identifiable by column nr). For the original colunm names please check this [source](https://opendatasus.saude.gov.br/dataset/b772ee55-07cd-44d8-958f-b12edd004e0b/resource/38ead83d-b115-4219-852e-7244792bc311/download/dicionario-de-dados-vacinacao.pdf).

To have the most updated dataset up to 31 August 2021, I downloaded the file from [OpenDataSus](https://opendatasus.saude.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8?view_id=6bf433e7-16ec-4dfa-a725-08db02322bd6) on 02 Sep 21 at 07 am, and re-named my downloaded master file to match this (sc020921_07am.csv).


used tip from this [link](https://www.geeksforgeeks.org/pandas-strip-whitespace-from-entire-dataframe/) to clear whitespace 

In [6]:
sc0209_07am = pd.read_csv('/content/drive/MyDrive/Alura DS Bootcamp/Combinations of Concern Projects/Vac_C19_CoC_5_sc0209_07am/sc020921_07am.csv', sep =';', header = 0, skipinitialspace = True, names=['c1_document_id','c2_p_id','c3_p_idade','c4_p_dn','c5_p_sex','c6_p_r_codigo','c7_p_r_valor','c8_p_ibgemunicipio','c9_p_pais','c10_p_nmmunicipio','c11_p_endereco_nmpais','c12_p_uf','c13_p_cep','c14_p_enumnacionalidade','c15_e_valor','c16_e_razaosocial','c17_e_nofantasia','c18_e_municipio_codigo','c19_e_municipio_nome', 'c20_e_uf', 'c21_v_grupoatendimento_codigo','c22_v_grupoatendimento_nome','c23_v_categoria_codigo','c24_v_categoria_nome','c25_v_lote', 'c26_v_fabricante_nome','c27_v_fabricante_referencia','c28_v_dataaplicacao','c29_v_descricao_dose','c30_v_codigo','c31_v_nome','c32_sistema_origem','c33_data_importacao_rnds','c34_id_sistema_origem'])

In [7]:
sc0209_07am.shape

(6822321, 34)

#<font color = 'brown'>IMPORTANT REMINDERS BEFORE WE MOVE ON: 

 

1. For information: 1.967.992 new registers were added between 04 Aug 21 (4.854.329 registrations) and 02 Sep 21. 

2. Each row, or register, correspond to the application of one dose, either Dose 1 or Dose 2. Therefore the total of 6.822.321 registrations in this dataset does not mean 6.822.321 vaccinated people in Santa Catarina.

3.   Per oficial [DIVE's boletim published on 01 Sep 21](http://www.dive.sc.gov.br/conteudos/boletim2021/corona57/corona57.pdf), up to 28 August the distribution betweeen doses 1 and dose 2 were:

*   dose 1 -      4.223.939
*   dose 2 -      1.806.959
*   Single dose -   221.850
*   Total -       6.252.748 (so, not as updated as our master dataset sc0209_07am)

4. Acording to the national central records for covid-19 distribution consulted on 02 Sep 2021 at 04 pm, the Brazilian state of Santa Catarina received from the Federal government to this date a total of 7.938.290 doses.

5. The four covid19 vaccines used in Santa Catarina (SC) until 02 Sep 21 are:

| Vaccine <br>Manufacturers 	| Nr of <br>doses 	| Actual <br>doses interval 	| Interval changed <br>during<br> the vaccination plan <br>from 	| Vaccine available<br> in SC since? 	|
|:-:	|:-:	|:-:	|:-:	|:-:	|
| Coronavac_Butantan 	| 2 	| 21 to 28 days 	| 14 Days 	| 18 Jan 21 	|
| AztraZeneca 	| 2 	| 120 days 	| 28 days 	| 23 Jan 21 	|
| Pfizer 	| 2 	| 120 days 	| No 	| 02 May 21 	|
| Janssen 	| 1 	| NA 	| NA 	| 24 Jun 21 	|


The above table was easily created with a tool from this [source](https://www.tablesgenerator.com/markdown_tables) as suggested by tip from this [one](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)

For further information about the change of the vacination intervals [Folha de Sao Paulo - 05 May 2021](https://www1.folha.uol.com.br/equilibrioesaude/2021/05/veja-quais-sao-os-intervalos-entre-as-doses-das-vacinas-contra-covid-no-brasil-e-por-que-eles-sao-diferentes.shtml)

In [8]:
sc0209_07am.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6822321 entries, 0 to 6822320
Data columns (total 34 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   c1_document_id                 object 
 1   c2_p_id                        object 
 2   c3_p_idade                     float64
 3   c4_p_dn                        object 
 4   c5_p_sex                       object 
 5   c6_p_r_codigo                  float64
 6   c7_p_r_valor                   object 
 7   c8_p_ibgemunicipio             float64
 8   c9_p_pais                      float64
 9   c10_p_nmmunicipio              object 
 10  c11_p_endereco_nmpais          object 
 11  c12_p_uf                       object 
 12  c13_p_cep                      float64
 13  c14_p_enumnacionalidade        object 
 14  c15_e_valor                    int64  
 15  c16_e_razaosocial              object 
 16  c17_e_nofantasia               object 
 17  c18_e_municipio_codigo         int64  
 18  c1

In [9]:
sc0209_07am.head()

,c1_document_id,c2_p_id,c3_p_idade,c4_p_dn,c5_p_sex,c6_p_r_codigo,c7_p_r_valor,c8_p_ibgemunicipio,c9_p_pais,c10_p_nmmunicipio,c11_p_endereco_nmpais,c12_p_uf,c13_p_cep,c14_p_enumnacionalidade,c15_e_valor,c16_e_razaosocial,c17_e_nofantasia,c18_e_municipio_codigo,c19_e_municipio_nome,c20_e_uf,c21_v_grupoatendimento_codigo,c22_v_grupoatendimento_nome,c23_v_categoria_codigo,c24_v_categoria_nome,c25_v_lote,c26_v_fabricante_nome,c27_v_fabricante_referencia,c28_v_dataaplicacao,c29_v_descricao_dose,c30_v_codigo,c31_v_nome,c32_sistema_origem,c33_data_importacao_rnds,c34_id_sistema_origem
0,8abd775e-0b0d-4b0c-b9a2-e61529d9e629-i0b0,e950263bfdb40a5e7b351def93b16de705648e9a0cedf9...,41.0,1980-04-08,M,99.0,SEM INFORMACAO,431490.0,10.0,PORTO ALEGRE,BRASIL,RS,90520.0,B,7032218,MUNICIPIO DE CRICIUMA,UNIDADE BASICA DE SAUDE CENTRO,420460,CRICIUMA,SC,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,FA9095,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-06,1ª Dose,87,Vacina covid-19 - BNT162b2 - BioNTech/Fosun Ph...,Novo PNI,2021-07-06T15:28:41.000Z,16341
1,9e2c6ed0-af55-4e72-aa79-f704780eb34b-i0b0,93ba0606fc9ff53b616a35be59e4adaddffa148a7822fa...,22.0,1998-05-05,F,1.0,BRANCA,420410.0,10.0,CAXAMBU DO SUL,BRASIL,SC,89880.0,B,2537745,MUNICIPIO DE CHAPECO,CENTRO DE SAUDE DA FAMILIA NORTE,420420,CHAPECO,SC,926,Outros,9.0,Trabalhadores de Saúde,200279,FUNDACAO BUTANTAN,Organization/61189445000156,2021-03-04,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-03-05T10:54:23.000Z,16341
2,d27610a5-c219-406d-9be9-5620c13907e5-i0b0,f73cb61dcdb860f39cfdf99c1b1da4655528912c71c6de...,36.0,1985-01-12,M,1.0,BRANCA,420890.0,10.0,JARAGUA DO SUL,BRASIL,SC,89254.0,B,2745178,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,CENTRAL DE IMUNIZACAO,420890,JARAGUA DO SUL,SC,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,C202106107,FUNDACAO BUTANTAN,NaN,2021-07-22,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Saudetech,2021-07-22T19:31:40.000Z,17961
3,2f2c77b2-688e-4428-944d-20c3db3e5553-i0b0,085843e2bc46f9b133062a411c6411a1afc43dd9f36a5f...,33.0,1987-07-15,F,1.0,BRANCA,420130.0,10.0,ARAQUARI,BRASIL,SC,89245.0,B,2511681,JOINVILLE SECRETARIA DA SAUDE,UBSF BUCAREIN,420910,JOINVILLE,SC,923,Técnico de Enfermagem,9.0,Trabalhadores de Saúde,214VCD057Z,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-05-29,2ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-05-29T21:38:16.000Z,16341
4,80ea7e85-a765-45fa-88cc-07ea09239476-i0b0,4a3f52a247ba87dec7d4c076bd40cdeccc93b09a43ad3b...,39.0,1982-05-22,F,1.0,BRANCA,420900.0,10.0,JOACABA,BRASIL,SC,89600.0,B,2560658,PREFEITURA MUNICIPAL DE JOACABA,ESF CENTRO 1,420900,JOACABA,SC,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,EY0586,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-05,1ª Dose,87,Vacina covid-19 - BNT162b2 - BioNTech/Fosun Ph...,Novo PNI,2021-07-05T17:43:04.000Z,16341


## <Font color = 'green'>*Converted the 3 date's columns from object to datetime.*

Tips for the conversion of c33 taken from this [link](https://www.statology.org/convert-datetime-to-date-pandas/)

In [10]:
sc0209_07am['c4_p_dn'] = pd.to_datetime(sc0209_07am['c4_p_dn'])
sc0209_07am['c28_v_dataaplicacao'] = pd.to_datetime(sc0209_07am['c28_v_dataaplicacao'])
sc0209_07am['c33_data_importacao_rnds'] = pd.to_datetime(sc0209_07am['c33_data_importacao_rnds']).dt.date
sc0209_07am['c33_data_importacao_rnds'] = pd.to_datetime(sc0209_07am['c33_data_importacao_rnds']).dt.normalize()

In [11]:
sc0209_07am.tail(3)

,c1_document_id,c2_p_id,c3_p_idade,c4_p_dn,c5_p_sex,c6_p_r_codigo,c7_p_r_valor,c8_p_ibgemunicipio,c9_p_pais,c10_p_nmmunicipio,c11_p_endereco_nmpais,c12_p_uf,c13_p_cep,c14_p_enumnacionalidade,c15_e_valor,c16_e_razaosocial,c17_e_nofantasia,c18_e_municipio_codigo,c19_e_municipio_nome,c20_e_uf,c21_v_grupoatendimento_codigo,c22_v_grupoatendimento_nome,c23_v_categoria_codigo,c24_v_categoria_nome,c25_v_lote,c26_v_fabricante_nome,c27_v_fabricante_referencia,c28_v_dataaplicacao,c29_v_descricao_dose,c30_v_codigo,c31_v_nome,c32_sistema_origem,c33_data_importacao_rnds,c34_id_sistema_origem
6822318,3e2d30aa-49cc-4f75-a9eb-cbb46833185f-i0b0,6a2cde3792443fb4d61df5a29c4db905c004142b0972fc...,69.0,1952-01-06,M,1.0,BRANCA,421170.0,10.0,ORLEANS,BRASIL,SC,88870.0,B,2513846,MUNICIPIO DE ORLEANS,UNIDADE SAO LUCAS,421170,ORLEANS,SC,203,Pessoas de 70 a 74 anos,2.0,Faixa Etária,210131,FUNDACAO BUTANTAN,Organization/61189445000156,2021-04-21,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-04-28,16341
6822319,df5f1392-2831-4f62-9ce8-e90174d87d56-i0b0,8c97e84edad03f1dcd12b6f61f1d9516edcfb2ce618d9d...,87.0,1934-05-07,F,1.0,BRANCA,420420.0,10.0,CHAPECO,BRASIL,SC,89804.0,B,2537729,MUNICIPIO DE CHAPECO,CENTRO DE SAUDE DA FAMILIA CRISTO REI,420420,CHAPECO,SC,205,Pessoas de 80 anos ou mais,2.0,Faixa Etária,213VCD031W,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-05-25,2ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-05-25,16341
6822320,454d9c46-e972-463d-8fb8-4a5f521a1782-i0b0,cb6ce369b46afe8b64c9d5d3080cb4cc764120d91fa5d5...,52.0,1969-04-07,M,1.0,BRANCA,420820.0,10.0,ITAJAI,BRASIL,SC,88310.0,B,7072414,PREFEITURA MUNICIPAL DE CAMBORIU,POLICLINICA MUNICIPAL,420320,CAMBORIU,SC,103,Diabetes Mellitus,1.0,Comorbidades,215VCD147W,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-08-03,2ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-08-03,16341


In [12]:
sc0209_07am.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6822321 entries, 0 to 6822320
Data columns (total 34 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   c1_document_id                 object        
 1   c2_p_id                        object        
 2   c3_p_idade                     float64       
 3   c4_p_dn                        datetime64[ns]
 4   c5_p_sex                       object        
 5   c6_p_r_codigo                  float64       
 6   c7_p_r_valor                   object        
 7   c8_p_ibgemunicipio             float64       
 8   c9_p_pais                      float64       
 9   c10_p_nmmunicipio              object        
 10  c11_p_endereco_nmpais          object        
 11  c12_p_uf                       object        
 12  c13_p_cep                      float64       
 13  c14_p_enumnacionalidade        object        
 14  c15_e_valor                    int64         
 15  c16_e_razaosoci

##<font color = 'green'>*Eliminated empty space and shortned the code for variable column 'c29_v_descricao_dose'*

In [13]:
sc0209_07am.loc[sc0209_07am['c29_v_descricao_dose'] == '    1ª Dose', 'c29_v_descricao_dose'] = 'D1'
sc0209_07am.loc[sc0209_07am['c29_v_descricao_dose'] == '    2ª Dose', 'c29_v_descricao_dose'] = 'D2'
sc0209_07am.head()

,c1_document_id,c2_p_id,c3_p_idade,c4_p_dn,c5_p_sex,c6_p_r_codigo,c7_p_r_valor,c8_p_ibgemunicipio,c9_p_pais,c10_p_nmmunicipio,c11_p_endereco_nmpais,c12_p_uf,c13_p_cep,c14_p_enumnacionalidade,c15_e_valor,c16_e_razaosocial,c17_e_nofantasia,c18_e_municipio_codigo,c19_e_municipio_nome,c20_e_uf,c21_v_grupoatendimento_codigo,c22_v_grupoatendimento_nome,c23_v_categoria_codigo,c24_v_categoria_nome,c25_v_lote,c26_v_fabricante_nome,c27_v_fabricante_referencia,c28_v_dataaplicacao,c29_v_descricao_dose,c30_v_codigo,c31_v_nome,c32_sistema_origem,c33_data_importacao_rnds,c34_id_sistema_origem
0,8abd775e-0b0d-4b0c-b9a2-e61529d9e629-i0b0,e950263bfdb40a5e7b351def93b16de705648e9a0cedf9...,41.0,1980-04-08,M,99.0,SEM INFORMACAO,431490.0,10.0,PORTO ALEGRE,BRASIL,RS,90520.0,B,7032218,MUNICIPIO DE CRICIUMA,UNIDADE BASICA DE SAUDE CENTRO,420460,CRICIUMA,SC,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,FA9095,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-06,1ª Dose,87,Vacina covid-19 - BNT162b2 - BioNTech/Fosun Ph...,Novo PNI,2021-07-06,16341
1,9e2c6ed0-af55-4e72-aa79-f704780eb34b-i0b0,93ba0606fc9ff53b616a35be59e4adaddffa148a7822fa...,22.0,1998-05-05,F,1.0,BRANCA,420410.0,10.0,CAXAMBU DO SUL,BRASIL,SC,89880.0,B,2537745,MUNICIPIO DE CHAPECO,CENTRO DE SAUDE DA FAMILIA NORTE,420420,CHAPECO,SC,926,Outros,9.0,Trabalhadores de Saúde,200279,FUNDACAO BUTANTAN,Organization/61189445000156,2021-03-04,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-03-05,16341
2,d27610a5-c219-406d-9be9-5620c13907e5-i0b0,f73cb61dcdb860f39cfdf99c1b1da4655528912c71c6de...,36.0,1985-01-12,M,1.0,BRANCA,420890.0,10.0,JARAGUA DO SUL,BRASIL,SC,89254.0,B,2745178,PREFEITURA MUNICIPAL DE JARAGUA DO SUL,CENTRAL DE IMUNIZACAO,420890,JARAGUA DO SUL,SC,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,C202106107,FUNDACAO BUTANTAN,NaN,2021-07-22,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Saudetech,2021-07-22,17961
3,2f2c77b2-688e-4428-944d-20c3db3e5553-i0b0,085843e2bc46f9b133062a411c6411a1afc43dd9f36a5f...,33.0,1987-07-15,F,1.0,BRANCA,420130.0,10.0,ARAQUARI,BRASIL,SC,89245.0,B,2511681,JOINVILLE SECRETARIA DA SAUDE,UBSF BUCAREIN,420910,JOINVILLE,SC,923,Técnico de Enfermagem,9.0,Trabalhadores de Saúde,214VCD057Z,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-05-29,2ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-05-29,16341
4,80ea7e85-a765-45fa-88cc-07ea09239476-i0b0,4a3f52a247ba87dec7d4c076bd40cdeccc93b09a43ad3b...,39.0,1982-05-22,F,1.0,BRANCA,420900.0,10.0,JOACABA,BRASIL,SC,89600.0,B,2560658,PREFEITURA MUNICIPAL DE JOACABA,ESF CENTRO 1,420900,JOACABA,SC,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,EY0586,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-05,1ª Dose,87,Vacina covid-19 - BNT162b2 - BioNTech/Fosun Ph...,Novo PNI,2021-07-05,16341


## <Font color = 'green'>*Sorting the entire dataset based on dates of the variable 'c28_v_dataaplicacao'*

In [14]:
sc0209_07am.sort_values(by='c28_v_dataaplicacao', ascending=True, inplace=True)
sc0209_07am

,c1_document_id,c2_p_id,c3_p_idade,c4_p_dn,c5_p_sex,c6_p_r_codigo,c7_p_r_valor,c8_p_ibgemunicipio,c9_p_pais,c10_p_nmmunicipio,c11_p_endereco_nmpais,c12_p_uf,c13_p_cep,c14_p_enumnacionalidade,c15_e_valor,c16_e_razaosocial,c17_e_nofantasia,c18_e_municipio_codigo,c19_e_municipio_nome,c20_e_uf,c21_v_grupoatendimento_codigo,c22_v_grupoatendimento_nome,c23_v_categoria_codigo,c24_v_categoria_nome,c25_v_lote,c26_v_fabricante_nome,c27_v_fabricante_referencia,c28_v_dataaplicacao,c29_v_descricao_dose,c30_v_codigo,c31_v_nome,c32_sistema_origem,c33_data_importacao_rnds,c34_id_sistema_origem
2341431,c0622be3-b4a1-4e4f-9c21-8e0e0cc7f7da-i0b0,8ba3755c79ba0f8bbb14ac74e74116c9b2b4cd9ba13cb6...,35.0,1985-03-17,F,3.0,PARDA,421660.0,10.0,SAO JOSE,BRASIL,SC,88106.0,B,2622726,PREF MUN FLORIANOPOLIS SMS FMS,CS CANASVIEIRAS,420540,FLORIANOPOLIS,SC,905,Cuidador de Idosos,9.0,Trabalhadores de Saúde,202010036,SERUM INSTITUTE OF INDIA LTD,NaN,2020-12-21,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-04-25,18602
1746232,4af998fc-ba82-40e6-bfd4-d2fcde965c29-i0b0,537c73cbf959a7ade3fe86d87aa7ab9f8b1de3ad55a0b0...,55.0,1965-10-26,F,1.0,BRANCA,421660.0,10.0,SAO JOSE,BRASIL,SC,88103.0,B,2622726,PREF MUN FLORIANOPOLIS SMS FMS,CS CANASVIEIRAS,420540,FLORIANOPOLIS,SC,923,Técnico de Enfermagem,9.0,Trabalhadores de Saúde,202010036,SERUM INSTITUTE OF INDIA LTD,NaN,2020-12-21,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-04-26,18602
2474106,d041ac6e-fdfd-4875-944e-6d2c4cc79191-i0b0,6e5a239adbcd1b2cea00f85873bc3d12521eb58564d489...,37.0,1983-05-18,M,1.0,BRANCA,421500.0,10.0,RIO NEGRINHO,BRASIL,SC,89295.0,B,2510987,FUNDO MUNICIPAL DE SAUDE DE RIO NEGRINHO,CENTRO INTEGRADO DE SAUDE,421500,RIO NEGRINHO,SC,0,NaN,NaN,NaN,202010036,SERUM INSTITUTE OF INDIA LTD,NaN,2020-12-22,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-03-12,18602
6419100,dffd702e-cd88-4e7c-8795-e4e0a7784af2-i0b0,4c569cc933658e66d31439b75f16bb6465acbf8e516674...,74.0,1946-12-05,M,1.0,BRANCA,420380.0,10.0,CANOINHAS,BRASIL,SC,89460.0,B,9522611,MUNICIPIO DE CANOINHAS,UNIDADE BASICA DE SAUDE ALVINO MUHLMANN,420380,CANOINHAS,SC,203,Pessoas de 70 a 74 anos,2.0,Faixa Etária,210131,SINOVAC/BUTANTA,NaN,2021-01-10,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,IDS Saúde,2021-01-10,18562
6702305,818361b6-9747-435d-8bad-5d079290656f-i0b0,3717784b0535b1db0d866d510059a48dbc486892b1770a...,46.0,1974-03-02,F,99.0,SEM INFORMACAO,421660.0,10.0,SAO JOSE,BRASIL,SC,NaN,B,2663805,SECRETARIA MUNICIPAL DE SAUDE DE SAO JOSE,CENTRO DE SAUDE PICADAS DO SUL,421660,SAO JOSE,SC,0,NaN,NaN,NaN,200279,SERUM INSTITUTE OF INDIA LTD,NaN,2021-01-11,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-02-22,18602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294794,9bd9c022-735a-419b-81c2-e504fcb8de9e-i0b0,5413c66d3d0a847cf2cf6a256dedbafeb55df5c09babdc...,22.0,1999-01-07,M,1.0,BRANCA,420230.0,10.0,BIGUACU,BRASIL,SC,88160.0,B,116912,MUNICIPIO DE BIGUACU,CENTRAL MUNICIPAL DE REDE DE FRIO DE BIGUACU,420230,BIGUACU,SC,201,Pessoas de 60 a 64 anos,2.0,Faixa Etária,210396,FUNDACAO BUTANTAN,Organization/61189445000156,2021-08-30,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-08-30,16341
294790,7e9c0723-5aa7-43d7-b02b-c1bda72611c1-i0b0,a67aa706e42d40a520eb00cea14ce80761acee51ed4b9f...,24.0,1997-04-20,M,2.0,PRETA,420420.0,10.0,CHAPECO,BRASIL,SC,89809.0,E,2555476,PREFEITURA MUNICIPAL DE SEARA,POSTO DE SAUDE BAIRRO SAO JOAO,421750,SEARA,SC,1601,Trabalhadores Industriais,16.0,Trabalhadores Industriais,210362,FUNDACAO BUTANTAN,Organization/61189445000156,2021-08-30,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-08-30,16341
294788,ba479266-ba43-4b68-9685-2bb8a3358e49-i0b0,2f2a0b7b61bf9e3016ee60d5708f73d446a0eac488ba96...,18.0,2003-04-29,M,1.0,BRANCA,420420.0,10.0,CHAPECO,BRASIL,SC,89816.0,B,2537583,MUNICIPI

# <Font color ='blue'> **Phase 2 - Adding a new column in preparation for the dataset for analysis**

# <Font color = 'green'>**Step 3** _ Adding a column identifying the UDVE of each register (row) in the dataset.

SC has 295 counties. [DIVE](http://www.dive.sc.gov.br), the local state instituition in charge of the distribution and tracking of the covid19 vaccines applications, shared the 295 in 17 UDVEs (Decentralized Epidemiological Surveillance Units in english).

Usually DIVE refers to its UDVEs by the name of the main city in the region an UDVE covers. 

To facilitate the coding for this project I renamed the UDVEs with a number, where 01 starts on the most western side of the state of SC and double digits ones the eastern (or costal) regions. 

These numbers tried to match the number used for regionales by another state institution helping coordinate the covid19 vaccinations in the state, called [COSENS SC](https://www.cosemssc.org.br/regioes_de_saude/). 

Tried because as you may notice, DIVE is organised in 17 UDVE regions and COSEMS SC in 16 regions and their regions borders not always match. How that helps, I do not know.

<Font color = 'green'>Based on tip from this [source](https://stackoverflow.com/questions/63558830/how-to-create-a-new-column-in-pandas-and-add-value-to-that-new-column-based-on-t) managed to succesfully create a new column labelled 'c35_SC_UDVE_Nr', where each register got an identification for its UDVE based on the county code where the vaccine shot was applied.

In [15]:
def get_udve(row):
    if row ['c18_e_municipio_codigo'] in [420080,420208,420209,420215,420490,420500,420535,420640,420660,420765,420840,421100,421200,421223,421415,421507,421520,421555,421625,421670,421720,421875]:
        return 'udve01'
    elif row['c18_e_municipio_codigo'] in [420010,420253,420445,420517,420530,420560,420768,420917,420945,421055,421185,421227,421340,421610,421910,421950,421970]:
        return 'udve02'
    elif row['c18_e_municipio_codigo'] in [420050,420055,420165,420257,420310,420350,420410,420420,420435,420440,420470,420475,420543,420665,420775,420785,420895,421050,421090,421140,421145,421165,421187,421210,421290,421315,421420,421535,421568,421569,421575,421600,421690,421715,421730,421755,421775,421795,421885]:
        return 'udve03'
    elif row['c18_e_municipio_codigo'] in [420075,420127,420430,420760,420770,420780,420800,420985,421260,421310,421390,421750,421960]:
        return 'udve04'
    elif row['c18_e_municipio_codigo'] in [420005,420040,420287,420360,420390,420400,420415,420520,420670,420680,420860,420900,420920,421003,421105,421180,421790,421850,421915,421917,421985]:
        return 'udve05'
    elif row['c18_e_municipio_codigo'] in [420160,420300,420315,420480,420550,420555,420675,420757,420970,421005,421070,421300,421335,421440,421540,421550,421605,421825,421930]:
        return 'udve06'
    elif row['c18_e_municipio_codigo'] in [420213,420330,420380,420790,420810,421010,421030,421110,421220,421360,421500,421580,421830]:
        return 'udve07'
    elif row['c18_e_municipio_codigo'] in [420450,420650,420890,421060,421740]:
        return 'udve08'
    elif row['c18_e_municipio_codigo'] in [420130,420205,420210,420580,420845,420910,421620,421635]:
        return 'udve09'
    elif row['c18_e_municipio_codigo'] in [420200,420245,420320,420710,420820,420830,421000,421130,421250,421280,421350]:
        return 'udve10'
    elif row['c18_e_municipio_codigo'] in [420020,420030,420180,420190,420285,420419,420510,420690,420740,420850,420915,420950,420990,421085,421270,421370,421400,421410,421450,421460,421480,421530,421567,421780,421860,421920,421935,421940]:
        return 'udve11'
    elif row['c18_e_municipio_codigo'] in [420125,420170,420220,420240,420270,420290,420515,420590,420630,420750,421320,421470,421510,421820]:
        return 'udve12'
    elif row['c18_e_municipio_codigo'] in [420060,420070,420090,420110,420120,420230,420370,420540,420570,420600,420980,421020,421150,421190,421230,421430,421570,421590,421630,421660,421725,421800]:
        return 'udve13'
    elif row['c18_e_municipio_codigo'] in [420100,420243,420250,420260,420325,420340,420417,420455,420930,421175,421189,421205,421330,421505,421650,421680,421890,421895]:
        return 'udve14'
    elif row['c18_e_municipio_codigo'] in [420150,420280,420395,420610,420620,420720,420730,420880,420940,421240,421265,421490,421545,421560,421700,421710,421840,421870]:
        return 'udve15'
    elif row['c18_e_municipio_codigo'] in [420425,420460,420545,420700,420960,421120,421160,421170,421760,421835,421900,42200]:
        return 'udve16'
    elif row['c18_e_municipio_codigo'] in [420140,420195,420207,420519,420870,421040,421080,421125,421225,421380,421565,421640,421770,421810,421880]:
        return 'udve17'


In [16]:
sc0209_07am['c35_SC_UDVE_Nr'] = sc0209_07am.apply(get_udve, axis = 1)

In [17]:
sc0209_07am_udve = sc0209_07am

In [18]:
sc0209_07am_udve.head()

,c1_document_id,c2_p_id,c3_p_idade,c4_p_dn,c5_p_sex,c6_p_r_codigo,c7_p_r_valor,c8_p_ibgemunicipio,c9_p_pais,c10_p_nmmunicipio,c11_p_endereco_nmpais,c12_p_uf,c13_p_cep,c14_p_enumnacionalidade,c15_e_valor,c16_e_razaosocial,c17_e_nofantasia,c18_e_municipio_codigo,c19_e_municipio_nome,c20_e_uf,c21_v_grupoatendimento_codigo,c22_v_grupoatendimento_nome,c23_v_categoria_codigo,c24_v_categoria_nome,c25_v_lote,c26_v_fabricante_nome,c27_v_fabricante_referencia,c28_v_dataaplicacao,c29_v_descricao_dose,c30_v_codigo,c31_v_nome,c32_sistema_origem,c33_data_importacao_rnds,c34_id_sistema_origem,c35_SC_UDVE_Nr
2341431,c0622be3-b4a1-4e4f-9c21-8e0e0cc7f7da-i0b0,8ba3755c79ba0f8bbb14ac74e74116c9b2b4cd9ba13cb6...,35.0,1985-03-17,F,3.0,PARDA,421660.0,10.0,SAO JOSE,BRASIL,SC,88106.0,B,2622726,PREF MUN FLORIANOPOLIS SMS FMS,CS CANASVIEIRAS,420540,FLORIANOPOLIS,SC,905,Cuidador de Idosos,9.0,Trabalhadores de Saúde,202010036,SERUM INSTITUTE OF INDIA LTD,NaN,2020-12-21,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-04-25,18602,udve13
1746232,4af998fc-ba82-40e6-bfd4-d2fcde965c29-i0b0,537c73cbf959a7ade3fe86d87aa7ab9f8b1de3ad55a0b0...,55.0,1965-10-26,F,1.0,BRANCA,421660.0,10.0,SAO JOSE,BRASIL,SC,88103.0,B,2622726,PREF MUN FLORIANOPOLIS SMS FMS,CS CANASVIEIRAS,420540,FLORIANOPOLIS,SC,923,Técnico de Enfermagem,9.0,Trabalhadores de Saúde,202010036,SERUM INSTITUTE OF INDIA LTD,NaN,2020-12-21,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-04-26,18602,udve13
2474106,d041ac6e-fdfd-4875-944e-6d2c4cc79191-i0b0,6e5a239adbcd1b2cea00f85873bc3d12521eb58564d489...,37.0,1983-05-18,M,1.0,BRANCA,421500.0,10.0,RIO NEGRINHO,BRASIL,SC,89295.0,B,2510987,FUNDO MUNICIPAL DE SAUDE DE RIO NEGRINHO,CENTRO INTEGRADO DE SAUDE,421500,RIO NEGRINHO,SC,0,NaN,NaN,NaN,202010036,SERUM INSTITUTE OF INDIA LTD,NaN,2020-12-22,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-03-12,18602,udve07
6419100,dffd702e-cd88-4e7c-8795-e4e0a7784af2-i0b0,4c569cc933658e66d31439b75f16bb6465acbf8e516674...,74.0,1946-12-05,M,1.0,BRANCA,420380.0,10.0,CANOINHAS,BRASIL,SC,89460.0,B,9522611,MUNICIPIO DE CANOINHAS,UNIDADE BASICA DE SAUDE ALVINO MUHLMANN,420380,CANOINHAS,SC,203,Pessoas de 70 a 74 anos,2.0,Faixa Etária,210131,SINOVAC/BUTANTA,NaN,2021-01-10,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,IDS Saúde,2021-01-10,18562,udve07
6702305,818361b6-9747-435d-8bad-5d079290656f-i0b0,3717784b0535b1db0d866d510059a48dbc486892b1770a...,46.0,1974-03-02,F,99.0,SEM INFORMACAO,421660.0,10.0,SAO JOSE,BRASIL,SC,NaN,B,2663805,SECRETARIA MUNICIPAL DE SAUDE DE SAO JOSE,CENTRO DE SAUDE PICADAS DO SUL,421660,SAO JOSE,SC,0,NaN,NaN,NaN,200279,SERUM INSTITUTE OF INDIA LTD,NaN,2021-01-11,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-02-22,18602,udve13


In [19]:
sc0209_07am_udve.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6822321 entries, 2341431 to 2622695
Data columns (total 35 columns):
 #   Column                         Dtype         
---  ------                         -----         
 0   c1_document_id                 object        
 1   c2_p_id                        object        
 2   c3_p_idade                     float64       
 3   c4_p_dn                        datetime64[ns]
 4   c5_p_sex                       object        
 5   c6_p_r_codigo                  float64       
 6   c7_p_r_valor                   object        
 7   c8_p_ibgemunicipio             float64       
 8   c9_p_pais                      float64       
 9   c10_p_nmmunicipio              object        
 10  c11_p_endereco_nmpais          object        
 11  c12_p_uf                       object        
 12  c13_p_cep                      float64       
 13  c14_p_enumnacionalidade        object        
 14  c15_e_valor                    int64         
 15  c16_e_raz

#<font color = 'blue'>**Phase 3 - Identifying and formally defining 'Combinations of Concern' (CoC)**

# <Font Color='green'>**Step 4** - Identifying and counting the different combinations between the 4 variables that allow identication of which covid19 vaccine an individual received.

| Column name                               | Column name in English    | comment                       |
|-------------|-----------------------------|---------------------------|
|<Font color ='brown'>c26_v_fabricante_nome | c26_vaccine manufacturer  | 
|<Font color ='brown'>c30_v_codigo          | c30_v_code                |
|<Font color ='brown'>c31_v_nome            | c31_v_name                |
|<Font color ='brown'>c32_sistema_origem    | c32_source_system         | system used at local level to enter registration data


The process used in this step was done based on the learning gathered from this [source](https://stackoverflow.com/questions/35268817/unique-combinations-of-values-in-selected-columns-in-pandas-data-frame-and-count)

In [20]:
v_fabri_codigo_nome_sysorig_sc0209_07am_udve = sc0209_07am_udve[['c26_v_fabricante_nome','c30_v_codigo','c31_v_nome', 'c32_sistema_origem',]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6822321 entries, 2341431 to 2622695
Data columns (total 4 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   c26_v_fabricante_nome  object
 1   c30_v_codigo           int64 
 2   c31_v_nome             object
 3   c32_sistema_origem     object
dtypes: int64(1), object(3)
memory usage: 260.3+ MB


In [21]:
v_fabri_codigo_nome_sysorig_sc0209_07am_udve = sc0209_07am_udve[['c26_v_fabricante_nome','c30_v_codigo','c31_v_nome','c32_sistema_origem']]
v_fabri_codigo_nome_sysorig_sc0209_07am_udve

,c26_v_fabricante_nome,c30_v_codigo,c31_v_nome,c32_sistema_origem
2341431,SERUM INSTITUTE OF INDIA LTD,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE)
1746232,SERUM INSTITUTE OF INDIA LTD,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE)
2474106,SERUM INSTITUTE OF INDIA LTD,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE)
6419100,SINOVAC/BUTANTA,86,Covid-19-Coronavac-Sinovac/Butantan,IDS Saúde
6702305,SERUM INSTITUTE OF INDIA LTD,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE)
...,...,...,...,...
294794,FUNDACAO BUTANTAN,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI
294790,FUNDACAO BUTANTAN,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI
294788,FUNDACAO BUTANTAN,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI
294812,FUNDACAO BUTANTAN,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI


In [22]:
v_fabri_codigo_nome_sysorig_sc0209_07am_udve.groupby(['c26_v_fabricante_nome','c30_v_codigo','c31_v_nome','c32_sistema_origem']).size()

c26_v_fabricante_nome                    c30_v_codigo  c31_v_nome                                                 c32_sistema_origem               
ASTRAZENECA AB                           85            Vacina Covid-19 - Covishield                               G-MUS                                   5207
                                                                                                                  G-MUS - Gestão Municipal de Saúde      13524
                                                                                                                  G-Mus Inovadora Sistemas                 124
ASTRAZENECA FARMACEUTICA DO BRASIL LTDA  85            Vacina Covid-19 - Covishield                               IPM istemas LTDA                        5186
BUTANTAN                                 86            Covid-19-Coronavac-Sinovac/Butantan                        IPM istemas LTDA                         424
BioNTech/Fosun Pharma/Pfizer             87            Va

# <Font color = 'green'>**Step 5** - Defining and creating a new column ['c_36_Combination_of_Concern'] that identify and classify 'combinations of concern' based on their frequency.

#<Font color = 'brown'>**Definition of Combination of Concern(CoC)'**: within an individual vaccination registration, any combination between the 4 variables above mentioned that would raise doubt about which covid vaccine an individual received. 

Per example: 'SERUM INSTITUTE OF INDIA LTD' is ONE OF the manufacturers across the globe for AztraZeneca(AZ) covid 19 related vaccines. AZ covid19 vaccines produced by SERUM are known as ['COVISHIELD'](https://www.bbc.com/news/health-57695835).  That's inconsistent with a 'c31_v_nome' = 'Covid-19-Coronavac-Sinovac/Butantan' in the same register, as this last one is manufactured by Coronavac China and/or Butantan Brasil.

For more details about identification of covid19 vaccine manufacturers and vaccine names please check references at these [ANVISA source]((https://www.gov.br/anvisa/pt-br/assuntos/paf/coronavirus/vacinas)) re-accessed on 26 Jun 2021.

# <font color ='brown'> Coding 'Combinations of Concern' based on their frequency.

*   <Font color = 'brown'>**CoC_1** = Very High Frequency - more than 100,000 registers
*   <Font color = 'brown'>**CoC_2** = High Frequency - between 10,000 and 100,000 registers
*   <Font color = 'brown'>**CoC_3** = 'Low' Frequency - less than 10,000 registers
*   <Font color = 'brown'>**'no_issue'** = At best knowledge of this author no remarkble issue identified at the time this project was prepared ( 30 June 2021)

<font color = 'brown'>**Author's note:** Strictly speaking, from the Pharmacovigilance point of view, 'no issue' is not strictly correct. But it will fulfill for the moment the purpose of this part of the work. More details in the conclusion at the end.

The set of codes here after I was inspired by tips from this [source](https://towardsdatascience.com/the-ultimate-guide-for-column-creation-with-pandas-dataframes-83b8c565110e)

In [23]:
def combination_of_concern(row):
    if row['c26_v_fabricante_nome'] == 'SERUM INSTITUTE OF INDIA LTD' and row['c30_v_codigo'] == 86 and row['c31_v_nome'] == 'Covid-19-Coronavac-Sinovac/Butantan':
        return 'CoC_1'
    elif row['c26_v_fabricante_nome'] == 'SINOVAC LIFE SCIENCE CO LTD' and row['c30_v_codigo'] == 85 and row['c31_v_nome'] == 'Vacina Covid-19 - Covishield':
        return 'CoC_1'
    elif row['c26_v_fabricante_nome'] == 'MINISTERIO DA SAUDE' and row['c30_v_codigo'] == 89 and row['c31_v_nome'] == 'Covid-19-AstraZeneca ':
        return 'CoC_1'
    elif row['c26_v_fabricante_nome'] == 'SINOVAC/BUTANTA' and row['c30_v_codigo'] == 85 and row['c31_v_nome'] == 'Vacina Covid-19 - Covishield':
        return 'CoC_3'
    elif row['c26_v_fabricante_nome'] == 'SINOVAC/BUTANTA' and row['c30_v_codigo'] == 88 and row['c31_v_nome'] == 'Vacina Covid-19 - Covishield':
        return 'CoC_3'
    elif row['c26_v_fabricante_nome'] == 'FUNDACAO OSWALDO CRUZ' and row['c30_v_codigo'] == 86 and row['c31_v_nome'] == 'Covid-19-Coronavac-Sinovac/Butantan':
        return 'CoC_3'
    elif row['c26_v_fabricante_nome'] == 'FUNDACAO OSWALDO CRUZ' and row['c30_v_codigo'] == 88 and row['c31_v_nome'] == 'Vacina covid-19 - Ad26.COV2.S - Janssen-Cilag':
        return 'CoC_3'
    elif row['c26_v_fabricante_nome'] == 'OXFORD-ASTRAZENECA' and row['c30_v_codigo'] == 86 and row['c31_v_nome'] == 'Covid-19-Coronavac-Sinovac/Butantan':
        return 'CoC_3'
    elif row['c26_v_fabricante_nome'] == 'OXFORD-ASTRAZENECA' and row['c30_v_codigo'] == 87 and row['c31_v_nome'] == 'Vacina covid-19 - BNT162b2 - BioNTech/Fosun Ph...':
        return 'CoC_3'
    elif row['c26_v_fabricante_nome'] == 'PFIZER/BIONTECH' and row['c30_v_codigo'] == 86 and row['c31_v_nome'] == 'Covid-19-Coronavac-Sinovac/Butantan':
        return 'CoC_3'
    elif row['c26_v_fabricante_nome'] == 'INTERVAX LTDA' and row['c30_v_codigo'] == 85 and row['c31_v_nome'] == 'Vacina Covid-19 - Covishield':
        return 'CoC_3'
    else:
        return 'no_issue'
     

#<font color = 'brown'> When compared to last previous update on 04 Aug 2021, we notice the following changes in the 'Combination of Concern' categories:

*   <font color = 'blue'> A significant increase of CoC_1 frequency. 
*   <font color = 'blue'> No CoC_2 group in August 21 (there was 2 in May);
*   <font color = 'blue'> A small reduction on the nr of CoC_3 category.

The data above may suggest that clean up of the database is not happening in systematic and large scale for the CoC_1 category.







In [24]:
sc0209_07am_udve['c_36_Combination_of_Concern'] = sc0209_07am_udve.apply(combination_of_concern, axis=1)

In [25]:
sc0209_07am_udve_CoC = sc0209_07am_udve

In [26]:
sc0209_07am_udve_CoC.c_36_Combination_of_Concern.unique()

array(['CoC_1', 'no_issue', 'CoC_3'], dtype=object)

In [45]:
sc0209_07am_udve_CoC

,c1_document_id,c2_p_id,c3_p_idade,c4_p_dn,c5_p_sex,c6_p_r_codigo,c7_p_r_valor,c8_p_ibgemunicipio,c9_p_pais,c10_p_nmmunicipio,c11_p_endereco_nmpais,c12_p_uf,c13_p_cep,c14_p_enumnacionalidade,c15_e_valor,c16_e_razaosocial,c17_e_nofantasia,c18_e_municipio_codigo,c19_e_municipio_nome,c20_e_uf,c21_v_grupoatendimento_codigo,c22_v_grupoatendimento_nome,c23_v_categoria_codigo,c24_v_categoria_nome,c25_v_lote,c26_v_fabricante_nome,c27_v_fabricante_referencia,c28_v_dataaplicacao,c29_v_descricao_dose,c30_v_codigo,c31_v_nome,c32_sistema_origem,c33_data_importacao_rnds,c34_id_sistema_origem,c35_SC_UDVE_Nr,c_36_Combination_of_Concern
2341431,c0622be3-b4a1-4e4f-9c21-8e0e0cc7f7da-i0b0,8ba3755c79ba0f8bbb14ac74e74116c9b2b4cd9ba13cb6...,35.0,1985-03-17,F,3.0,PARDA,421660.0,10.0,SAO JOSE,BRASIL,SC,88106.0,B,2622726,PREF MUN FLORIANOPOLIS SMS FMS,CS CANASVIEIRAS,420540,FLORIANOPOLIS,SC,905,Cuidador de Idosos,9.0,Trabalhadores de Saúde,202010036,SERUM INSTITUTE OF INDIA LTD,NaN,2020-12-21,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-04-25,18602,udve13,CoC_1
1746232,4af998fc-ba82-40e6-bfd4-d2fcde965c29-i0b0,537c73cbf959a7ade3fe86d87aa7ab9f8b1de3ad55a0b0...,55.0,1965-10-26,F,1.0,BRANCA,421660.0,10.0,SAO JOSE,BRASIL,SC,88103.0,B,2622726,PREF MUN FLORIANOPOLIS SMS FMS,CS CANASVIEIRAS,420540,FLORIANOPOLIS,SC,923,Técnico de Enfermagem,9.0,Trabalhadores de Saúde,202010036,SERUM INSTITUTE OF INDIA LTD,NaN,2020-12-21,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-04-26,18602,udve13,CoC_1
2474106,d041ac6e-fdfd-4875-944e-6d2c4cc79191-i0b0,6e5a239adbcd1b2cea00f85873bc3d12521eb58564d489...,37.0,1983-05-18,M,1.0,BRANCA,421500.0,10.0,RIO NEGRINHO,BRASIL,SC,89295.0,B,2510987,FUNDO MUNICIPAL DE SAUDE DE RIO NEGRINHO,CENTRO INTEGRADO DE SAUDE,421500,RIO NEGRINHO,SC,0,NaN,NaN,NaN,202010036,SERUM INSTITUTE OF INDIA LTD,NaN,2020-12-22,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-03-12,18602,udve07,CoC_1
6419100,dffd702e-cd88-4e7c-8795-e4e0a7784af2-i0b0,4c569cc933658e66d31439b75f16bb6465acbf8e516674...,74.0,1946-12-05,M,1.0,BRANCA,420380.0,10.0,CANOINHAS,BRASIL,SC,89460.0,B,9522611,MUNICIPIO DE CANOINHAS,UNIDADE BASICA DE SAUDE ALVINO MUHLMANN,420380,CANOINHAS,SC,203,Pessoas de 70 a 74 anos,2.0,Faixa Etária,210131,SINOVAC/BUTANTA,NaN,2021-01-10,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,IDS Saúde,2021-01-10,18562,udve07,no_issue
6702305,818361b6-9747-435d-8bad-5d079290656f-i0b0,3717784b0535b1db0d866d510059a48dbc486892b1770a...,46.0,1974-03-02,F,99.0,SEM INFORMACAO,421660.0,10.0,SAO JOSE,BRASIL,SC,NaN,B,2663805,SECRETARIA MUNICIPAL DE SAUDE DE SAO JOSE,CENTRO DE SAUDE PICADAS DO SUL,421660,SAO JOSE,SC,0,NaN,NaN,NaN,200279,SERUM INSTITUTE OF INDIA LTD,NaN,2021-01-11,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,ESUS APS - NACIONAL (OFFLINE),2021-02-22,18602,udve13,CoC_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294794,9bd9c022-735a-419b-81c2-e504fcb8de9e-i0b0,5413c66d3d0a847cf2cf6a256dedbafeb55df5c09babdc...,22.0,1999-01-07,M,1.0,BRANCA,420230.0,10.0,BIGUACU,BRASIL,SC,88160.0,B,116912,MUNICIPIO DE BIGUACU,CENTRAL MUNICIPAL DE REDE DE FRIO DE BIGUACU,420230,BIGUACU,SC,201,Pessoas de 60 a 64 anos,2.0,Faixa Etária,210396,FUNDACAO BUTANTAN,Organization/61189445000156,2021-08-30,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-08-30,16341,udve13,no_issue
294790,7e9c0723-5aa7-43d7-b02b-c1bda72611c1-i0b0,a67aa706e42d40a520eb00cea14ce80761acee51ed4b9f...,24.0,1997-04-20,M,2.0,PRETA,420420.0,10.0,CHAPECO,BRASIL,SC,89809.0,E,2555476,PREFEITURA MUNICIPAL DE SEARA,POSTO DE SAUDE BAIRRO SAO JOAO,421750,SEARA,SC,1601,Trabalhadores Industriais,16.0,Trabalhadores Industriais,210362,FUNDACAO BUTANTAN,Organization/61189445000156,2021-08-30,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-08-30,16341,udve04,no_issue
294788,ba479266-ba43-4b68-9685-2bb8

# <Font color ='blue'>**Phase 4 - Crosstabing UDVE and CoC at county level**

# <Font color = 'green'>**Step 6** - Crosstabing UDVE and CoC

Tips from Alura curso 'Estatistica com Python parte 1: Frequencias e Medidas - aula 2 - Distribuicao de frequencia + for formating right - [source](https://stackoverflow.com/questions/17232013/how-to-set-the-pandas-dataframe-data-left-right-alignment) and [source](https://pbpython.com/styling-pandas.html)

In [28]:
udve_CoC = pd.crosstab(sc0209_07am_udve_CoC.c35_SC_UDVE_Nr , sc0209_07am_udve_CoC.c_36_Combination_of_Concern)
def highlight(s):
    if s.CoC_1 > 100000:
        return ['background-color: red'] * 3
    elif (s.CoC_1 > 10000) and (s.CoC_1 < 100000):
        return ['background-color: orange'] * 3
    elif (s.CoC_1 > 1000) and (s.CoC_1 < 10000):
        return ['background-color: yellow'] * 3
    else:
        return ['background-color: white'] * 3

udve_CoC.style.set_properties(**{'text-align': 'right'}).format('{0:,.0f}').apply(highlight, axis=1)


c_36_Combination_of_Concern,CoC_1,CoC_3,no_issue
c35_SC_UDVE_Nr,,,
udve01,801,0,"205,327"
udve02,"13,897",0,"132,596"
udve03,"21,240",0,"462,773"
udve04,46,0,"153,603"
udve05,"1,074",0,"200,399"
udve06,"36,221",0,"230,489"
udve07,"47,218",28,"262,479"
udve08,0,0,"237,516"
udve09,"47,827",1,"651,320"


Tip from [source](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html) (Finer Control: Display Values)

# <font color = 'green'>Proportions of CoCs categories per UDVE 

tip [source](https://stackoverflow.com/questions/50820659/compute-row-percentages-in-pandas-dataframe)

UDVEs 13, 7, 6, 9, 3 and 10 are, respectively, the ones of greatest concern due to the potential large cluster of unknown vaccination status (CoC1), based on the data of this dataset.

<Font color = 'red'>Percentual - a great example of how 'data cleaning' in clinical data sciences can be a very sensitive matter. CoC_3 is low in frequency but potentially high in consequencies ('No one is safe, until everyone is'). Low percentage does not mean 'low risk'.

In [29]:
Per_udve_CoC = udve_CoC[['CoC_1','CoC_3','no_issue']] = udve_CoC[['CoC_1','CoC_3','no_issue']].apply(lambda x: x/x.sum(), axis=1) * 1

Per_udve_CoC.style.set_properties(**{'text-align': 'right'}).format("{:.2%}")

c_36_Combination_of_Concern,CoC_1,CoC_3,no_issue
c35_SC_UDVE_Nr,,,
udve01,0.39%,0.00%,99.61%
udve02,9.49%,0.00%,90.51%
udve03,4.39%,0.00%,95.61%
udve04,0.03%,0.00%,99.97%
udve05,0.53%,0.00%,99.47%
udve06,13.58%,0.00%,86.42%
udve07,15.25%,0.01%,84.75%
udve08,0.00%,0.00%,100.00%
udve09,6.84%,0.00%,93.16%


# <Font color = 'green'>**Step 07** - Crosstabing UDVE and CoC, per county, for the two UDVES with the highest frequency of 'Combinations of Concern'(CoC): UDVEs 13 and 07 respectively.

# <font color ='red'>**UDVE 13**

In [30]:
udve_13 = sc0209_07am_udve_CoC.loc[sc0209_07am_udve_CoC['c35_SC_UDVE_Nr'] == 'udve13']
udve_13.shape


(1244747, 36)

In [31]:
udve_13_CoC = pd.crosstab(udve_13.c19_e_municipio_nome , udve_13.c_36_Combination_of_Concern)
udve_13_CoC.style.set_properties(**{'text-align': 'right'}).format('{0:,.0f}')

c_36_Combination_of_Concern,CoC_1,no_issue
c19_e_municipio_nome,,
AGUAS MORNAS,0,"7,284"
ALFREDO WAGNER,0,"9,683"
ANGELINA,0,"6,280"
ANITAPOLIS,765,"3,161"
ANTONIO CARLOS,688,"9,741"
BIGUACU,0,"66,925"
CANELINHA,0,"10,379"
FLORIANOPOLIS,"416,510","174,933"
GAROPABA,0,"28,024"


#<Font color = 'orange'>**UDVE 07**

In [32]:
udve_07 = sc0209_07am_udve_CoC.loc[sc0209_07am_udve_CoC['c35_SC_UDVE_Nr'] == 'udve07']
udve_07.shape

(309725, 36)

In [33]:
udve_07_CoC = pd.crosstab(udve_07.c19_e_municipio_nome , udve_07.c_36_Combination_of_Concern)
udve_07_CoC.style.set_properties(**{'text-align': 'right'}).format('{0:,.0f}')

c_36_Combination_of_Concern,CoC_1,CoC_3,no_issue
c19_e_municipio_nome,,,
BELA VISTA DO TOLDO,0,0,"5,330"
CAMPO ALEGRE,0,0,"12,076"
CANOINHAS,0,21,"42,181"
IRINEOPOLIS,0,0,"10,158"
ITAIOPOLIS,62,7,"19,789"
MAFRA,"42,738",0,"12,675"
MAJOR VIEIRA,33,0,"6,966"
MONTE CASTELO,342,0,"7,949"
PAPANDUVA,97,0,"18,169"


# <Font Color = 'blue'> **PHASE 5** - 'Time Serie' Exploration

#<Font color='green'> **Step 08** - Counting total nr of registers per month

Tips from [source](https://stackoverflow.com/questions/38792122/how-to-group-and-count-rows-by-month-and-year-using-pandas)

In [34]:
c_28 = sc0209_07am_udve_CoC['c28_v_dataaplicacao'].groupby([sc0209_07am_udve_CoC['c28_v_dataaplicacao'].dt.year.rename('year'), sc0209_07am_udve_CoC['c28_v_dataaplicacao'].dt.month.rename('month')]).agg({'count'})
c_28

count
year month         
2020 12           3
2021 1        67818
     2       169561
     3       595833
     4       805597
     5       750808
     6      1051602
     7      1644150
     8      1736949

# <Font Color = 'green'>**Step 9** - Checking 'Combination of Concern' frequency since start of covid19 vaccination in Santa Catarina as per dataset sc0209_07am.

Tips from [source](https://stackoverflow.com/questions/44908383/how-can-i-group-by-month-from-a-date-field-using-python-pandas)

In [35]:
c28_36 = sc0209_07am_udve_CoC.groupby([(sc0209_07am_udve_CoC['c28_v_dataaplicacao'].dt.strftime('%B')),'c_36_Combination_of_Concern']).size()
c28_36

c28_v_dataaplicacao  c_36_Combination_of_Concern
April                CoC_1                           117382
                     CoC_3                                8
                     no_issue                        688207
August               CoC_1                           134373
                     no_issue                       1602576
December             CoC_1                                3
February             CoC_1                            32456
                     CoC_3                               12
                     no_issue                        137093
January              CoC_1                            11061
                     CoC_3                                8
                     no_issue                         56749
July                 CoC_1                           163955
                     no_issue                       1480195
June                 CoC_1                           103511
                     no_issue                      

<Font color = 'red'> Here I could try to find out how to pivot the table above to the wished result here after (done manually). However, I enjoy this 'sensorial tête_à_tête' with the data as it provides very interesting insights. 

Per example we notice that: 


*   Wrong registrations entered for December 2020 are still not corrected (vaccination in SC only started on 18 Jan 21);
*   When we compare data from end June with end July we notice that frequency for category CoC_1 is still raising for the early months of January, February and March, but reducing for April, May and June.

In [36]:
sc0209_07am_c_28_c_36 = {
    'Month':    ['December','January','February', 'March', 'April','May','June','July', 'August'],
    'CoC_1':     [3, 11061, 32456, 116791, 117382, 87650, 103511, 163955, 134373],
    'CoC_2':     [0, 0, 0, 0, 0, 0, 0, 0, 0],
    'CoC_3':     [0, 8, 12, 3, 8, 1, 0, 0, 0],
    'no_issue':  [0, 56749, 137093, 479039, 688207, 663157, 103511, 1480195, 1602576]
    }

In [37]:
sc0209_07am_df_c_28_c_36 = pd.DataFrame(data=sc0209_07am_c_28_c_36)
sc0209_07am_df_c_28_c_36

,Month,CoC_1,CoC_2,CoC_3,no_issue
0,December,3,0,0,0
1,January,11061,0,8,56749
2,February,32456,0,12,137093
3,March,116791,0,3,479039
4,April,117382,0,8,688207
5,May,87650,0,1,663157
6,June,103511,0,0,103511
7,July,163955,0,0,1480195
8,August,134373,0,0,1602576


In [43]:
sc0209_07am_df_c_28_c_36.sum()

Month       DecemberJanuaryFebruaryMarchAprilMayJuneJulyAu...
CoC_1                                                  767182
CoC_2                                                       0
CoC_3                                                      32
no_issue                                              5210527
dtype: object

#<font color = 'green'>**Step 10** - Recalling data from 04 August 2021 and 02 July 21 for comparisons with data from up to 02 September 21:

In [38]:
sc0408_10am_c_28_c_36 = {
    'Month':    ['December','January','February', 'March', 'April','May','June','July'],
    'CoC_1':     [3, 12607, 34525, 127466, 121957, 81883, 92074,105555],
    'CoC_2':     [0, 0, 0, 0, 0, 0, 0, 0],
    'CoC_3':     [0, 9, 12, 3, 8, 1, 0, 0],
    'no_issue':  [0, 57701, 138245, 486487, 687464, 647836, 927710, 1322208]
    }

In [39]:
sc0408_10am_c_28_c_36 = pd.DataFrame(data=sc0408_10am_c_28_c_36)
sc0408_10am_c_28_c_36

,Month,CoC_1,CoC_2,CoC_3,no_issue
0,December,3,0,0,0
1,January,12607,0,9,57701
2,February,34525,0,12,138245
3,March,127466,0,3,486487
4,April,121957,0,8,687464
5,May,81883,0,1,647836
6,June,92074,0,0,927710
7,July,105555,0,0,1322208


In [41]:
sc_0207_07am_c_28_c_36 = {'Month':['December', 'January',  'February', 'March', 'April',     'May',    'Jun'],
                    'CoC_1':   [3,           11834,       34385,   112652,   103964,   58107,    47241],
                    'CoC_2':   [0,               0,           0,        0,        0,       0,        0],
                    'CoC_3':   [0,               9,          13,      437,        6,       2,        0],
                 'no_issue':   [0,           56505,      137148,   484289,   680737,  629570,   776442]}

In [42]:
sc_0207_07am_c_28_c_36 = pd.DataFrame(data=sc_0207_07am_c_28_c_36)
sc_0207_07am_c_28_c_36

,Month,CoC_1,CoC_2,CoC_3,no_issue
0,December,3,0,0,0
1,January,11834,0,9,56505
2,February,34385,0,13,137148
3,March,112652,0,437,484289
4,April,103964,0,6,680737
5,May,58107,0,2,629570
6,Jun,47241,0,0,776442


# **<Font color='blue'>Brief Comments and key conclusions:**

*   <font color = 'green'>**DES_EXCELEI !!!** </font>- With the help of the Alura Bootcamp DS 2, Aluras courses Programming with Python, Python for DataSciences, services as stackoverflow, medium and others, I accomplished my objective to deliver this module project entirely within one Google Colab notebook using Python related packages.

* At the end of August 2021 we still notice a 11% of registrations ( (CoC_1 + CoC_3 = 767.182 ) out of a total of 6.822.321)) classified as 'Combinations_of_Concern'. 

* In other words, based on the federal government open source database used for this work, in the state of Santa Catarina we have 767.182 registers (out of an total o 6.822.321) that we can not confirm which vacine an individual received.  

* This confirm the overall monthly trend around 12 % of Combinations of Concern identified since the vaccination plan started in the Brazilian tate of Santa Catarina last January. 

* UDVE 13 alone clusters 74% of the state total of 'Combinations of Concern' (568143 out of 767182).

* Most of them in Florianopolis, the capital of Santa Catarina state, where for 45% of the registers up to end of August 2021, based on the federal source we used for this work, we do not know which vacine an individual received.

*   "'*No_issue*'" - as already mentioned, from the pharmacovigilance point of view, is not totally correct, as I noticed some other 'combinations of worrying' (to distinguish of the definition of 'CoC'). 

This project is a work in progress and is being conceived and designed with the end-user in mind: the SC DIVE's UDVE managers and local vaccination teams accross the state of Santa Catarina. 

My final goal is that local UDVE teams may use the delivers of this data sciences project to help correct, clean and update their covid 19 registration files. 

I am also aiming a high level of automation of the entire monitoring process, so the end-users can, at any time, quickly and easily track progress of the corrective actions to address the 'Combinations of Concern' - CoC.

PS: To run the program for this 5th edition I was obliged to upgrade to a paying Google Colab Pro, to be able to use a GPU and high-RAM runtime! I suppose that was due to the increase of 1.967.992 news registers from previous month.
